In [ ]:
# Importing libraries needed for image processing, handling data, and plotting
import cv2  # for image processing
import pandas as pd  # to work with excel and csv files
import numpy as np  # to handle arrays
from PIL import Image as im  # to create image
import matplotlib.pyplot as plt  # For plotting graphs
from skimage.metrics import structural_similarity as ssim  # For comparing images using SSIM
import os  # for file operations
import csv # Used to save values in a CSV file

# reading the image in grayscale mode (2 stands for grayscale)
img = cv2.imread(r"C:\Users\Rounak Deb\Downloads\Final_Jupyter_project_10th_may\Image_1.jpg", 2)

img  # just checking the image matrix


In [ ]:
# applying binary thresholding to the image
bwimg = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

bwimg  # checking threshold output

# showing the binary image using OpenCV
cv2.imshow('Binary', bwimg[1])
cv2.waitKey(0)  # waits for any key press
cv2.destroyAllWindows()  # closes all OpenCV windows


In [ ]:
bwimg[1].shape  # checking the shape of the thresholded (binary) image


In [ ]:
a = []  # this will store pixel values in 1D

# storing only the first 100 rows into a
for i in range(100):
    a.extend(bwimg[1][i])  # flattening row-wise

a  # checking the list


In [ ]:
len(a)  # checking total number of elements

# converting all white pixels (255) to 1
for i in range(len(a)):
    if a[i] == 255:
        a[i] = 1

a  # verifying the modified list


In [ ]:
bitstream = pd.DataFrame(a)  # converting the list into a DataFrame

bitstream  # checking the data

bitstream.shape  # checking rows and columns


In [ ]:
# creating a folder to save csv and excel
os.makedirs("tx_rx_csv_files", exist_ok=True)

# saving the bitstream in excel and csv formats
bitstream.to_excel("tx_rx_csv_files/transmit1.xlsx", index=False, header=False)
bitstream.to_csv("tx_rx_csv_files/transmit1.csv", index=False, header=False)

In [ ]:
import pandas as pd

# Load the CSV without header
dataframe = pd.read_csv("outputs_tx_rx/received.csv", header=None)

# Access the first column (integer index, not string)
x = dataframe[0].tolist()

# Optionally insert a zero at the start
x.insert(0, 0)

print(x[:10])  # Print first 10 values to check


In [ ]:
# converting all 1s to 225 to reconstruct image
for i in range(len(x)):
    if x[i] == 1:
        x[i] = 225

a_arr1 = []  # will store pixels

# storing first 10000 values only
for i in range(10000):
    a_arr1.append(x[i])


In [ ]:
a_arr = np.array(a_arr1)  # converting list to array
a_arr.resize(100, 100)  # resizing into 100x100 image

a_arr.shape  # checking shape

x  # checking received list again

a_arr  # checking image array


In [ ]:
# Convert array to uint8 format if not already
image_array = np.uint8(a_arr)

# Show image in a separate popup window using OpenCV
cv2.imshow("Reconstructed Image", image_array)

# Wait for any key to be pressed
cv2.waitKey(0)

# Close all OpenCV windows
cv2.destroyAllWindows()


In [ ]:
# create folder to save images
os.makedirs("outputs_tx_rx", exist_ok=True)

# converting both TX and RX to proper image format
tx_image = im.fromarray(bwimg[1])  # original transmitting image
rx_image = im.fromarray(a_arr.astype(np.uint8))  # received image

# saving both images
tx_image.save("outputs_tx_rx/transmitting_image.jpg")
rx_image.save("outputs_tx_rx/receiving_image.jpg")


In [ ]:
# create new blank image of double width
combined = im.new('L', (tx_image.width + rx_image.width, tx_image.height))

# paste both images side-by-side
combined.paste(tx_image, (0, 0))
combined.paste(rx_image, (tx_image.width, 0))

# save comparison image
combined.save("outputs_tx_rx/tx_rx_comparison_image.jpg")

# show in notebook
display(combined)


In [ ]:
# Using the transmitting image (TX) as it is since it is already in grayscale
tx_image_gray = np.array(tx_image)

# Using the receiving image (RX) as it is since it is also grayscale
rx_image_gray = a_arr  # RX image is already in the required format

# Convert both images to uint8 to avoid any data type issues while calculating pixel differences
tx_image_gray = np.uint8(tx_image_gray)
rx_image_gray = np.uint8(rx_image_gray)

# Calculate Pearson's Correlation Coefficient
correlation_coefficient = np.corrcoef(tx_image_gray.flatten(), rx_image_gray.flatten())[0, 1]
print(f"Pearson Correlation Coefficient: {correlation_coefficient}")

# Calculate RMSE (Root Mean Squared Error)
rmse_value = np.sqrt(np.sum((tx_image_gray - rx_image_gray) ** 2) / tx_image_gray.size)
print(f"RMSE: {rmse_value}")

# Calculate PSNR (Peak Signal-to-Noise Ratio)
max_pixel_value = 255.0  # Maximum pixel value for uint8 images
psnr_value = 20 * np.log10(max_pixel_value / rmse_value)
print(f"PSNR: {psnr_value} dB")

# Calculate Normalized Cross-Correlation (NCC)
norm_tx = tx_image_gray.flatten() - np.mean(tx_image_gray)
norm_rx = rx_image_gray.flatten() - np.mean(rx_image_gray)
ncc_value = np.sum(norm_tx * norm_rx) / np.sqrt(np.sum(norm_tx**2) * np.sum(norm_rx**2))
print(f"Normalized Cross-Correlation (NCC): {ncc_value}")

# Calculate SSIM (Structural Similarity Index)
ssim_value, _ = ssim(tx_image_gray, rx_image_gray, full=True)
print(f"SSIM: {ssim_value}")


In [ ]:
# Flatten the grayscale images
tx_flat = tx_image_gray.flatten()
rx_flat = rx_image_gray.flatten()

# Linear fit: RX = m × TX + c
m, c = np.polyfit(tx_flat, rx_flat, 1)

# Create the figure
plt.figure(figsize=(8, 6), dpi=300)  # High DPI for clear export

# Scatter plot of RX vs TX
plt.scatter(tx_flat, rx_flat, s=1, alpha=0.3, color='blue', label='Pixel Data (RX vs TX)')

# Best-fit line
plt.plot(tx_flat, m * tx_flat + c, color='red', linewidth=2, label=f'Fit: RX = {m:.2f} × TX + {c:.2f}')

# Labels and Title
plt.xlabel('Transmitted Pixel Intensity (TX)', fontsize=12)
plt.ylabel('Received Pixel Intensity (RX)', fontsize=12)
plt.title('Linear Relationship Between Transmitted and Received Images', fontsize=14)

# Grid and legend
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(fontsize=10)

# Save the figure
plt.tight_layout()
plt.savefig('outputs_tx_rx/linear_fit_rx_vs_tx_presentable.jpg')
plt.show()


In [ ]:
# Save all metrics and linear fit values to a CSV
comparison_data = {
    'Pearson Correlation Coefficient': [correlation_coefficient],
    'RMSE': [rmse_value],
    'PSNR (dB)': [psnr_value],
    'Normalized Cross-Correlation (NCC)': [ncc_value],
    'SSIM': [ssim_value],
    'Linear Fit Slope (m)': [m],
    'Linear Fit Intercept (c)': [c]
}

# Create a DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Save to CSV in output folder
comparison_df.to_csv("outputs_tx_rx/comparison_parameters.csv", index=False)


In [ ]:
# Convert image pixel values to range between 0 and 1
tx_normalized = tx_image_gray / 255.0
rx_normalized = rx_image_gray / 255.0

# Make the 2D images into 1D lists
tx_flat_normalized = tx_normalized.flatten()
rx_flat_normalized = rx_normalized.flatten()

# Find average pixel values
mean_tx_intensity = np.mean(tx_flat_normalized)
mean_rx_intensity = np.mean(rx_flat_normalized)

# Fit a line: RX = m × TX + c
m_normalized, c_normalized = np.polyfit(tx_flat_normalized, rx_flat_normalized, 1)

# Draw the scatter points and the fitted line
plt.figure(figsize=(8, 6), dpi=300)
plt.scatter(tx_flat_normalized, rx_flat_normalized, s=1, alpha=0.3, color='blue', label='Pixel Data')
plt.plot(tx_flat_normalized, m_normalized * tx_flat_normalized + c_normalized, color='red', linewidth=2,
         label=f'Line: RX = {m_normalized:.2f} × TX + {c_normalized:.2f}')

# Add labels and title
plt.xlabel('TX Pixel (Normalized)')
plt.ylabel('RX Pixel (Normalized)')
plt.title('TX vs RX Pixel Comparison')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Save the graph
plt.savefig('outputs_tx_rx/normalized_intensity_comparison.jpg')
plt.show()

# Save slope, intercept, and averages in a CSV file
with open('outputs_tx_rx/parameters.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Parameter", "Value"])
    writer.writerow(["Slope (m)", f"{m_normalized:.4f}"])
    writer.writerow(["Intercept (c)", f"{c_normalized:.4f}"])
    writer.writerow(["Mean TX", f"{mean_tx_intensity:.4f}"])
    writer.writerow(["Mean RX", f"{mean_rx_intensity:.4f}"])
